<a href="https://www.kaggle.com/code/ranjeetshrivastav/tps-aug-plotly-pycaret-lda?scriptVersionId=104572661" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

<h1 align="center" style="font-weight: bold">TPS August 2022</h1>

In [1]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings('ignore')

import plotly.express as px
import plotly.graph_objects as go

from sklearn.preprocessing import LabelEncoder
from sklearn.impute import SimpleImputer

In [2]:
train = pd.read_csv(r'../input/tabular-playground-series-aug-2022/train.csv')
train.head()

,id,product_code,loading,attribute_0,attribute_1,attribute_2,attribute_3,measurement_0,measurement_1,measurement_2,...,measurement_9,measurement_10,measurement_11,measurement_12,measurement_13,measurement_14,measurement_15,measurement_16,measurement_17,failure
0,0,A,80.10,material_7,material_8,9,5,7,8,4,...,10.672,15.859,17.594,15.193,15.029,NaN,13.034,14.684,764.100,0
1,1,A,84.89,material_7,material_8,9,5,14,3,3,...,12.448,17.947,17.915,11.755,14.732,15.425,14.395,15.631,682.057,0
2,2,A,82.43,material_7,material_8,9,5,12,1,5,...,12.715,15.607,NaN,13.798,16.711,18.631,14.094,17.946,663.376,0
3,3,A,101.07,material_7,material_8,9,5,13,2,6,...,12.471,16.346,18.377,10.020,15.250,15.562,16.154,17.172,826.282,0
4,4,A,188.06,material_7,material_8,9,5,9,2,8,...,10.337,17.082,19.932,12.428,16.182,12.760,13.153,16.412,579.885,0


In [3]:
test = pd.read_csv(r'../input/tabular-playground-series-aug-2022/test.csv')
test.head()

,id,product_code,loading,attribute_0,attribute_1,attribute_2,attribute_3,measurement_0,measurement_1,measurement_2,...,measurement_8,measurement_9,measurement_10,measurement_11,measurement_12,measurement_13,measurement_14,measurement_15,measurement_16,measurement_17
0,26570,F,119.57,material_5,material_6,6,4,6,9,6,...,18.654,10.802,15.909,18.070,13.772,13.659,16.825,13.742,17.710,634.612
1,26571,F,113.51,material_5,material_6,6,4,11,8,0,...,19.368,12.032,13.998,NaN,12.473,17.468,16.708,14.776,14.102,537.037
2,26572,F,112.16,material_5,material_6,6,4,8,12,4,...,17.774,11.743,17.046,18.086,10.907,13.363,15.737,17.065,16.021,658.995
3,26573,F,112.72,material_5,material_6,6,4,8,11,10,...,18.948,11.790,18.165,16.163,10.933,15.501,15.667,12.620,16.111,594.301
4,26574,F,208.00,material_5,material_6,6,4,14,16,8,...,19.141,12.370,14.578,17.849,11.941,16.070,16.183,13.324,17.150,801.044


In [4]:
sub = pd.read_csv(r'../input/tabular-playground-series-aug-2022/sample_submission.csv')
sub.head()

,id,failure
0,26570,0.0
1,26571,0.0
2,26572,0.0
3,26573,0.0
4,26574,0.0


In [5]:
print(f'train set have {train.shape[0]} rows and {train.shape[1]} columns.')
print(f'test set have {test.shape[0]} rows and {test.shape[1]} columns.') 
print(f'sample_submission set have {sub.shape[0]} rows and {sub.shape[1]} columns.') 

train set have 26570 rows and 26 columns.
test set have 20775 rows and 25 columns.
sample_submission set have 20775 rows and 2 columns.


In [6]:
train.drop('id',axis=1,inplace=True)
test.drop('id',axis=1,inplace=True)

In [7]:
train.dtypes

product_code       object
loading           float64
attribute_0        object
attribute_1        object
attribute_2         int64
attribute_3         int64
measurement_0       int64
measurement_1       int64
measurement_2       int64
measurement_3     float64
measurement_4     float64
measurement_5     float64
measurement_6     float64
measurement_7     float64
measurement_8     float64
measurement_9     float64
measurement_10    float64
measurement_11    float64
measurement_12    float64
measurement_13    float64
measurement_14    float64
measurement_15    float64
measurement_16    float64
measurement_17    float64
failure             int64
dtype: object

In [8]:
train.nunique()

product_code          5
loading           11950
attribute_0           2
attribute_1           3
attribute_2           4
attribute_3           4
measurement_0        29
measurement_1        30
measurement_2        25
measurement_3      4721
measurement_4      4692
measurement_5      4671
measurement_6      4704
measurement_7      4734
measurement_8      4713
measurement_9      4708
measurement_10     6177
measurement_11     6526
measurement_12     6392
measurement_13     5271
measurement_14     6389
measurement_15     6577
measurement_16     7035
measurement_17    23612
failure               2
dtype: int64

In [9]:
train.isnull().sum()

product_code         0
loading            250
attribute_0          0
attribute_1          0
attribute_2          0
attribute_3          0
measurement_0        0
measurement_1        0
measurement_2        0
measurement_3      381
measurement_4      538
measurement_5      676
measurement_6      796
measurement_7      937
measurement_8     1048
measurement_9     1227
measurement_10    1300
measurement_11    1468
measurement_12    1601
measurement_13    1774
measurement_14    1874
measurement_15    2009
measurement_16    2110
measurement_17    2284
failure              0
dtype: int64

In [10]:
train['m3_missing']=train.measurement_3.isnull().astype(int)
train['m5_missing']=train.measurement_5.isnull().astype(int)

test['m3_missing']=test.measurement_3.isnull().astype(int)
test['m5_missing']=test.measurement_5.isnull().astype(int)

In [11]:
s = SimpleImputer(missing_values = np.nan, strategy = 'median')
var = ['loading','measurement_3','measurement_4','measurement_5','measurement_6','measurement_7','measurement_8','measurement_9','measurement_10',
       'measurement_11','measurement_12','measurement_13','measurement_14','measurement_15','measurement_16','measurement_17']
train[var] = s.fit_transform(train[var])
test[var] = s.transform(test[var])

In [12]:
train['loading'] = np.log(train['loading'])

In [13]:
train.describe().T

,count,mean,std,min,25%,50%,75%,max
loading,26570.0,4.806226,0.296601,3.501344,4.607268,4.807213,5.002855,5.955475
attribute_2,26570.0,6.754046,1.471852,5.000000,6.000000,6.000000,8.000000,9.000000
attribute_3,26570.0,7.240459,1.456493,5.000000,6.000000,8.000000,8.000000,9.000000
measurement_0,26570.0,7.415883,4.116690,0.000000,4.000000,7.000000,10.000000,29.000000
measurement_1,26570.0,8.232518,4.199401,0.000000,5.000000,8.000000,11.000000,29.000000
measurement_2,26570.0,6.256568,3.309109,0.000000,4.000000,6.000000,8.000000,24.000000
measurement_3,26570.0,17.791463,0.993996,13.968000,17.127000,17.787000,18.456000,21.499000
measurement_4,26570.0,11.732008,0.985948,8.008000,11.072000,11.733000,12.396000,16.484000
measurement_5,26570.0,17.127910,0.983657,12.073000,16.466000,17.132000,17.785000,21.425000
measurement_6,26570.0,17.510916,0.980947,12.715000,16.865000,17.516000,18.155750,21.543000


<h2 align="center" style="font-weight: bold">Data Visualization using Plotly</h2>

In [14]:
fig = go.Figure(data=[go.Pie(labels=train.product_code, hole=.4)])
fig.add_annotation(text='Product Code',
                   x=0.5,y=0.5,showarrow=False,font_size=14,opacity=0.7,font_family='monospace')
fig.update_traces(hoverinfo='label+percent+value',
                  marker=dict(colors=['darkorange','blue'], line=dict(color='#000000', width=2)))
fig.update_layout(
    font_family='monospace',
    title=dict(text='Product Code',x=0.47,y=0.98,
               font=dict(color='black',size=20)),
    legend=dict(orientation='v',traceorder='reversed'),
    hoverlabel=dict(bgcolor='white'))
fig.update_traces(textposition='outside', textinfo='percent+label')
fig.show()

In [15]:
fig = go.Figure(data=[go.Pie(labels=train.attribute_0, hole=.4)])
fig.add_annotation(text='Attribute 0',
                   x=0.5,y=0.5,showarrow=False,font_size=14,opacity=0.7,font_family='monospace')
fig.update_traces(hoverinfo='label+percent+value',
                  marker=dict(colors=['darkorange','blue'], line=dict(color='#000000', width=2)))
fig.update_layout(
    font_family='monospace',
    title=dict(text='Attribute 0',x=0.47,y=0.98,
               font=dict(color='black',size=20)),
    legend=dict(orientation='v',traceorder='reversed'),
    hoverlabel=dict(bgcolor='white'))
fig.update_traces(textposition='outside', textinfo='percent+label')
fig.show()

In [16]:
fig = go.Figure(data=[go.Pie(labels=train.attribute_1, hole=.4)])
fig.add_annotation(text='Attribute 1',
                   x=0.5,y=0.5,showarrow=False,font_size=14,opacity=0.7,font_family='monospace')
fig.update_traces(hoverinfo='label+percent+value',
                  marker=dict(colors=['darkorange','blue'], line=dict(color='#000000', width=2)))
fig.update_layout(
    font_family='monospace',
    title=dict(text='Attribute 1',x=0.47,y=0.98,
               font=dict(color='black',size=20)),
    legend=dict(orientation='v',traceorder='reversed'),
    hoverlabel=dict(bgcolor='white'))
fig.update_traces(textposition='outside', textinfo='percent+label')
fig.show()

In [17]:
train['product_code'] = train['product_code'].replace({'A':0,
                                      'B':1,
                                      'C':2,
                                      'D':3,
                                      'E':4})

test['product_code'] = test['product_code'].replace({'F':5,
                                      'I':6,
                                      'G':7,
                                      'H':8})

train['attribute_0'] = train['attribute_0'].replace({'material_7':0,
                                      'material_5':1})

test['attribute_0'] = test['attribute_0'].replace({'material_7':0,
                                      'material_5':1})

train['attribute_1'] = train['attribute_1'].replace({'material_6':2,
                                      'material_5':1, 'material_8':3})

test['attribute_1'] = test['attribute_1'].replace({'material_7':0,
                                      'material_5':1, 'material_6':2})


In [18]:
from sklearn.preprocessing import StandardScaler
ss = StandardScaler()
var = ['measurement_0','measurement_1','measurement_2','measurement_3','measurement_4','measurement_5','measurement_6','measurement_7','measurement_8','measurement_9','measurement_10',
       'measurement_11','measurement_12','measurement_13','measurement_14','measurement_15','measurement_16','measurement_17']
train[var] = ss.fit_transform(train[var])
test[var] = ss.transform(test[var])

In [19]:
train.head()

,product_code,loading,attribute_0,attribute_1,attribute_2,attribute_3,measurement_0,measurement_1,measurement_2,measurement_3,...,measurement_11,measurement_12,measurement_13,measurement_14,measurement_15,measurement_16,measurement_17,failure,m3_missing,m5_missing
0,0,4.383276,0,3,9,5,-0.101025,-0.055370,-0.681939,0.250043,...,-1.069064,2.417899,-0.557589,-0.005457,-1.315581,-1.082400,0.533173,0,0,0
1,0,4.441356,0,3,9,5,1.599402,-1.246039,-0.984141,0.424092,...,-0.851904,0.035794,-0.823716,-0.433039,-0.401846,-0.504840,-0.162797,0,0,0
2,0,4.411949,0,3,9,5,1.113565,-1.722306,-0.379737,0.267146,...,0.025191,1.451338,0.949567,1.795950,-0.603929,0.907042,-0.321268,0,0,0
3,0,4.615813,0,3,9,5,1.356484,-1.484172,-0.077535,-0.499471,...,-0.539357,-1.166345,-0.359562,-0.337789,0.779094,0.434992,1.060662,0,0,0
4,0,5.236761,0,3,9,5,0.384811,-1.484172,0.526868,1.563957,...,0.512617,0.502099,0.475557,-2.285894,-1.235688,-0.028520,-1.029521,0,0,0


<h2 align="center" style="font-weight: bold">PyCaret</h2>

In [20]:
! pip install --ignore-installed --pre pycaret

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 544.5/544.5 kB 2.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.4/78.4 kB 6.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.2/11.2 MB 34.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 54.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 106.8/106.8 kB 8.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 63.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.8/24.8 MB 28.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.3/134.3 kB 11.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... - done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 307.0/307.0 kB 19.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.8/43.8 kB 3.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 15.7/15.7 MB 33.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 133.1/133.1 kB 9.2 MB/s e

In [21]:
from pycaret.classification import setup, compare_models, blend_models, finalize_model, predict_model, get_config

In [22]:
def pycaret_model(train, target,test, n_select, fold,opt):
    print('Setup Your Data....')
    setup(data=train,
          target=target)
  
    print('Comparing Models....')
    best = compare_models(sort = opt,n_select=n_select)
    
#   print('Blending Models....')
#   blended = blend_models(estimator_list= best, fold=fold, optimize=opt)

    prep_pipe = get_config("pipeline")
    prep_pipe.steps.append(['final_model', best])
    pred = prep_pipe.predict_proba(test)[:,1]
    
    return pred

In [23]:
result = pycaret_model(train,'failure',test, 1, 5, 'AUC')

Setup Your Data....


,Description,Value
0,Session id,3157
1,Target,failure
2,Target type,Binary
3,Original data shape,"(26570, 27)"
4,Transformed data shape,"(26570, 27)"
5,Transformed train set shape,"(18598, 27)"
6,Transformed test set shape,"(7972, 27)"
7,Numeric features,26
8,Preprocess,True
9,Imputation type,simple


Comparing Models....


,Model,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC,TT (Sec)
lr,Logistic Regression,0.7875,0.5860,0.0003,0.1000,0.0005,0.0004,0.0045,1.5190
lda,Linear Discriminant Analysis,0.7875,0.5860,0.0005,0.2000,0.0010,0.0008,0.0089,0.1050
gbc,Gradient Boosting Classifier,0.7867,0.5773,0.0030,0.3519,0.0060,0.0021,0.0128,6.1450
nb,Naive Bayes,0.7741,0.5752,0.0286,0.2487,0.0511,0.0060,0.0121,0.0580
ada,Ada Boost Classifier,0.7858,0.5667,0.0030,0.2238,0.0060,0.0003,0.0011,1.3350
lightgbm,Light Gradient Boosting Machine,0.7863,0.5577,0.0081,0.3998,0.0158,0.0071,0.0285,0.8150
catboost,CatBoost Classifier,0.7867,0.5558,0.0040,0.3171,0.0080,0.0032,0.0148,12.3420
et,Extra Trees Classifier,0.7873,0.5501,0.0005,0.2000,0.0010,0.0005,0.0053,2.1340
rf,Random Forest Classifier,0.7867,0.5462,0.0018,0.1683,0.0035,0.0006,0.0023,5.8000
xgboost,Extreme Gradient Boosting,0.7752,0.5354,0.0400,0.2877,0.0699,0.0199,0.0325,4.4840


Processing:   0%|          | 0/69 [00:00<?, ?it/s]

In [24]:
sub['failure'] = result
sub.to_csv('sub.csv',index=False)

<h2 align="center" style="font-weight: bold">Linear Discriminant Analysis (LDA)</h2>

In [25]:
X = train.drop('failure',axis=1)
y = train['failure']

In [26]:
from sklearn.model_selection import StratifiedKFold
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.metrics import roc_auc_score

folds = StratifiedKFold(n_splits=5, shuffle=True)

for fold, (trn_idx, val_idx) in enumerate(folds.split(X,y)):
    print(f"Fold: {fold}")
    X_train, X_test = X.iloc[trn_idx], X.iloc[val_idx]
    y_train, y_test = y.iloc[trn_idx], y.iloc[val_idx]

    model = LinearDiscriminantAnalysis(solver = 'lsqr')
   
    model.fit(X_train, y_train)
    y_pred = model.predict_proba(X_test)[:,1]
    roc = roc_auc_score(y_test, y_pred)
    
    print(f" roc_auc_score: {roc}")
    print("-"*50)

Fold: 0
 roc_auc_score: 0.60511760653479
--------------------------------------------------
Fold: 1
 roc_auc_score: 0.5855270393746087
--------------------------------------------------
Fold: 2
 roc_auc_score: 0.5844961420667016
--------------------------------------------------
Fold: 3
 roc_auc_score: 0.5956308482377028
--------------------------------------------------
Fold: 4
 roc_auc_score: 0.5802921792246908
--------------------------------------------------


In [27]:
sub['failure'] = model.predict_proba(test)[:,1]
sub.to_csv('lda.csv',index=False)